## Anomalous Access of the Curriculum

## Executive Summary

The goal of this project is to answer questions regarding accessing curriculum logs. The data was acquired from the curriculum logs database, cleaned and manipulated to answer questions. The questions were answered using a combination of pandas manipulation as well as anomaly detection techniques.

### Project Overview

- The tasks for our project were planned using this [Trello board](https://trello.com/b/qAGMpGzT/anomaly-detection-project)
- Python scripts were used to automate the process of acquiring and preparing the data
- The data was cleaned by converting dates to datetime objects, dropping duplicates, encoding new features, renaming columns to be more descriptive
- Outliers in the data were analyzed as well as the distributions of variables explored in depth

### Key Takeaways

- The square footage, location (latitude and longitude) and number of bedrooms were found to be the best drivers of tax value
- Some variables like number of bedrooms and bathrooms were highly correlated with each other 
- Age of home did not seem to have a significant impact on the tax value of the properties
- There were some outliers in the data in square footage and number of bedrooms and bathrooms 
    - These were handled by using a Robust Scaler to reduce the effect of outliers on the model's performance
- The baseline mean's RMSE of 620877USD was utilized as the RMSE to beat
    - All models beat this RMSE 
- The Root Mean Squared Error (RMSE) and R squared were utilized as metrics to evaluate a model's performance
    - My best model's RMSE was 463791USD and R squared was 44%.
    - This improved to an RMSE of 440857USD and R squared of 48% on the test split

### Data Dictionary

- You can find the data dictionary detailing all the variables used and created [here](https://github.com/mariamnaqvi/predicting_property_values/blob/main/README.md)

Through our analysis we aimed to answer the following questions. This notebook provides answers to each question and an explanation of how that conclusion was reached.

In [2]:
# import libraries and custom modules to use 
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
import math


import seaborn as sns
import acquire as a
import prepare as p
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [3]:
# acquire df
df = a.get_curriculum_data()

In [5]:
df.head()

,start_date,end_date,created_at,cohort,program_id,date,time,endpoint,user_id,ip
0,2015-09-22,2016-02-06,2016-06-14 19:52:26,Hampton,1.0,2018-01-26,09:55:03,/,1,97.105.19.61
1,2015-09-22,2016-02-06,2016-06-14 19:52:26,Hampton,1.0,2018-01-26,09:56:02,java-ii,1,97.105.19.61
2,2015-09-22,2016-02-06,2016-06-14 19:52:26,Hampton,1.0,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61
3,2015-09-22,2016-02-06,2016-06-14 19:52:26,Hampton,1.0,2018-01-26,09:56:06,slides/object_oriented_programming,1,97.105.19.61
4,2018-01-08,2018-05-17,2018-01-08 13:59:10,Teddy,2.0,2018-01-26,09:56:24,javascript-i/conditionals,2,97.105.19.61


In [6]:
# prepare data
df = p.prep_logs(df)

There are 4015 duplicate rows in your dataset - these will be dropped.
----------------


### 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

There are 4 different programs available in the dataset:
1. Full-Stack PHP Program
2. Full-Stack Java Program
3. Data Science
4. Front-End Program

We checked the value counts for each program's endpoints to see which lesson was referred to the most across cohorts. The following results were found:

- The most commonly accessed page was the home page for all cohorts
- For the PHP and Java program, the javascript-i lesson was accessed the most.
- For Data Science, the classification overview lesson was accessed the most.
- For the Front-End Program, the html-css lesson was accessed the most.

In [7]:
# check different programs offered
df.program_id.value_counts()

2.0    710307
3.0    102821
1.0     30432
4.0         5
Name: program_id, dtype: int64

In [8]:
df.groupby('program_id').endpoint.value_counts()

program_id  endpoint                                   
1.0         /                                              1680
            index.html                                     1006
            javascript-i                                    735
            html-css                                        540
            spring                                          496
                                                           ... 
3.0         where                                             1
4.0         content/html-css                                  2
            /                                                 1
            content/html-css/gitbook/images/favicon.ico       1
            content/html-css/introduction.html                1
Name: endpoint, Length: 3309, dtype: int64

In [9]:
df.endpoint[(df.program_id == 1)].value_counts()

/                                                1680
index.html                                       1006
javascript-i                                      735
html-css                                          540
spring                                            496
                                                 ... 
content/examples/html                               1
html-css/bootstrap-grid-system                      1
html/forms                                          1
2.0_Intro_Stats                                     1
content/examples/javascript/conditionals.html       1
Name: endpoint, Length: 710, dtype: int64

In [10]:
df.endpoint[(df.program_id == 2)].value_counts()

/                                      35701
javascript-i                           17392
toc                                    17320
search/search_index.json               15164
java-iii                               12621
                                       ...  
selectors                                  1
uploads/598dc43df39e2.jpg                  1
html-css/grid/css-grid-fundamentals        1
9_Appendix_TSAD_Lesson3                    1
student/120                                1
Name: endpoint, Length: 1913, dtype: int64

In [11]:
df.endpoint[(df.program_id == 3)].value_counts()

/                                           8235
search/search_index.json                    2187
classification/overview                     1783
1-fundamentals/1.1-intro-to-data-science    1632
1-fundamentals/modern-data-scientist.jpg    1613
                                            ... 
grades                                         1
12-distributed-ml/4-acquire                    1
imports                                        1
joins                                          1
nlp                                            1
Name: endpoint, Length: 682, dtype: int64

In [12]:
df.endpoint[(df.program_id == 4)].value_counts()

content/html-css                               2
content/html-css/introduction.html             1
content/html-css/gitbook/images/favicon.ico    1
/                                              1
Name: endpoint, dtype: int64

### 2. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [45]:
df.head()

,start_date,end_date,created_at,cohort,program_id,time,endpoint,user_id,ip,accessed_after,is_active,program_name
date,,,,,,,,,,,,
2018-01-26,2015-09-22,2016-02-06,2016-06-14 19:52:26,Hampton,1.0,09:55:03,/,1,97.105.19.61,1,False,Full Stack PHP
2018-01-26,2015-09-22,2016-02-06,2016-06-14 19:52:26,Hampton,1.0,09:56:02,java-ii,1,97.105.19.61,1,False,Full Stack PHP
2018-01-26,2015-09-22,2016-02-06,2016-06-14 19:52:26,Hampton,1.0,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,1,False,Full Stack PHP
2018-01-26,2015-09-22,2016-02-06,2016-06-14 19:52:26,Hampton,1.0,09:56:06,slides/object_oriented_programming,1,97.105.19.61,1,False,Full Stack PHP
2018-01-26,2018-01-08,2018-05-17,2018-01-08 13:59:10,Teddy,2.0,09:56:24,javascript-i/conditionals,2,97.105.19.61,0,True,Web Development


In [58]:
active_students = df[df.is_active==True]

In [59]:
active_students.groupby('cohort').user_id.nunique()

cohort
Andromeda     27
Apex          21
Bash          22
Bayes         22
Betelgeuse    20
Ceres         29
Curie         19
Darden        28
Deimos        25
Easley        17
Europa        28
Florence      21
Fortuna       28
Ganymede      28
Hyperion      23
Jupiter       33
Kalypso       23
Luna          20
Marco         31
Neptune       37
Oberon        26
Sequoia       21
Teddy         31
Ulysses       28
Voyageurs     33
Wrangell      26
Xanadu        20
Yosemite      27
Zion          25
Name: user_id, dtype: int64

In [65]:
active_students.endpoint.value_counts().idxmin()

'java-iii/servelet'

In [60]:
active_students.groupby('user_id')[(active_students.endpoint.value_counts().idxmin]

KeyError: "Column not found: <bound method Series.idxmin of /                                                                                                                                                                                                                                                                                                                         30216\ntoc                                                                                                                                                                                                                                                                                                                       14923\nsearch/search_index.json                                                                                                                                                                                                                                                                                  

### 3. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

- It appears that there might be some web scraping happening.
- The most frequently used ip address would be the ip address for the school.
- We explored the anomalies in number of page views beyond the upper bollinger band.
- User 341 had some suspicious activity with 272 pages viewed on 2019-03-03 and 104 pages viewed on 2020-04-21. 
- The user used a few different ip addresses on 03-03-2019 and the curriculum was accessed after 10pm to midnight.
- The user also accessed a wide range of topics. 
- On 2020-04-21, the user accessed a wide range of topics and viewed 104 pages using a different ip address. This was a while after the student had graduated from the Zion cohort in June 2019.


In [13]:
# most frequently used ip 
df.ip.value_counts().idxmax()

'97.105.19.58'

In [14]:
def prep(df, user):
    '''
    This function takes in a df and a single user. 
    It converts the date column to a datetime object and sets that as the index.
    It also resamples daily counts for number of page views.
    '''
    df = df[df.user_id == user]
    pages = df['endpoint'].resample('d').count()
    return pages

def compute_pct_b(pages, span, weight, user):
    '''
    This function takes in the pages, the number of days, the weight to calculate the standard deviation and the user.
    It calculates the mid, upper and lower bollinger bands and concatenates the upper and lower bands.
    It then creates a new dataframe with the number of page view, midband, upper and lower bands.
    It calculate the %b and adds a user id column to the df and then returns the df.
    '''
    midband = pages.ewm(span=span).mean()
    stdev = pages.ewm(span=span).std()
    ub = midband + stdev * weight
    lb = midband - stdev * weight
    bb = pd.concat([ub, lb], axis=1)
    my_df = pd.concat([pages, midband, bb], axis=1)
    my_df.columns = ['pages', 'midband', 'ub', 'lb']
    my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def find_anomalies(df, user, span, weight):
    '''
    This function takes in a df, user id, number of days as span and the weight to compute %b. 
    It then prepares the df by calling the prep function.
    It also creates a new dataframe by calling the compute pct b function.
    It returns a subset of the df where %b is greater than 1.
    '''
    pages = prep(df, user)
    my_df = compute_pct_b(pages, span, weight, user)
    return my_df[my_df.pct_b > 1]

In [15]:
# test on a single user

user = 68
span = 30
weight = 3

# initialize empty df
anomalies = pd.DataFrame()

# call function to create a new df with anomalies for the user 
user_df = find_anomalies(df, user, span, weight)

# concatenate the above df created with the empty anomalies df
anomalies = pd.concat([anomalies, user_df], axis=0)

In [16]:
anomalies

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-05-04,67,7.035731,60.259707,-46.188246,1.063320,68
2018-06-25,50,4.811412,45.265482,-35.642658,1.058517,68
2018-08-20,17,1.240542,15.590082,-13.108997,1.049128,68
2018-10-16,13,0.898678,10.970672,-9.173316,1.100741,68
2018-11-15,27,1.863468,22.364493,-18.637557,1.113055,68
2019-01-30,21,1.407805,17.354013,-14.538403,1.114321,68
2019-03-25,24,1.622558,19.932129,-16.687012,1.111086,68
2019-06-27,9,0.587115,7.439000,-6.264770,1.113910,68
2019-07-10,14,1.554444,13.962988,-10.854099,1.001491,68


In [17]:
# loop through all users to find anomalies for each
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [18]:
anomalies

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-01-29,44,3.623334,42.081344,-34.834677,1.024945,1
2019-07-08,101,8.224728,96.466493,-80.017037,1.025688,1
2018-10-15,9,0.581421,8.457413,-7.294571,1.034446,3
2019-01-09,4,0.262470,3.802115,-3.277175,1.027953,3
2019-04-06,2,0.129825,1.889712,-1.630061,1.031334,3
...,...,...,...,...,...,...
2021-01-24,7,0.574393,6.857226,-5.708440,1.011362,817
2021-03-25,59,6.797425,58.410273,-44.815423,1.005713,843
2021-02-23,67,8.229409,66.322904,-49.864086,1.005828,851


In [19]:
# look at value counts for pages in anomalies without sorting the counts
anomalies.pages.value_counts(sort = False)

1      49
2      35
3      71
4      76
5      60
       ..
165     1
179     1
192     1
197     1
272     1
Name: pages, Length: 95, dtype: int64

The highest number of page views is 272 so let's investigate that.

In [20]:
# look at a subset of anomalies df where the number of page views is equal to 272
anomalies[anomalies.pages == 272]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-03-03,272,24.721632,266.780128,-217.336864,1.010782,341


- Looks like it is user 341 who has a very high number of page views. We can investigate this user's activity further to check if there is anything suspicous about their IP address.

In [21]:
# pull a subset of data where user id is 341
anomalies[anomalies.user_id == 341]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-03-03,272,24.721632,266.780128,-217.336864,1.010782,341
2020-02-18,3,0.193548,2.817185,-2.430088,1.034840,341
2020-04-21,104,6.928145,97.719968,-83.863678,1.034585,341
2021-02-09,10,0.659630,9.508602,-8.189342,1.027766,341


- Looks like on 2019-03-03 and 2020-04-21 this user had a very number of page views. 

In [25]:
df.ip[df.user_id == 341].value_counts()

97.105.19.58       677
172.124.70.146     457
173.174.243.231    232
204.44.112.76      180
70.123.203.131      25
192.171.117.210      3
Name: ip, dtype: int64

- This user seems to be using a few different IP addresses to access the curriculum. We will look at a subset of the df to check the dates of 03-03 and 04-21 further.

In [33]:
sus_1 = df.loc['2019-03-02':'2019-03-04']

sus_2 = df.loc['2020-04-20':'2020-04-21']

In [31]:
# set to display all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [32]:
sus_1[sus_1.user_id == 341]

,start_date,end_date,created_at,cohort,program_id,time,endpoint,user_id,ip,accessed_after,is_active,program_name
date,,,,,,,,,,,,
2019-03-03,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,21:14:08,/,341,173.174.243.231,0,True,Web Development
2019-03-03,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,21:14:14,jquery,341,173.174.243.231,0,True,Web Development
2019-03-03,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,21:14:27,jquery/essential-methods/traversing,341,173.174.243.231,0,True,Web Development
2019-03-03,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,21:15:25,jquery/effects,341,173.174.243.231,0,True,Web Development
2019-03-03,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,22:52:05,html-css,341,204.44.112.76,0,True,Web Development
2019-03-03,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,22:52:06,javascript-i,341,204.44.112.76,0,True,Web Development
2019-03-03,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,22:52:06,java-i,341,204.44.112.76,0,True,Web Development
2019-03-03,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,22:52:06,java-ii,341,204.44.112.76,0,True,Web Development
2019-03-03,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,22:52:06,javascript-ii,341,204.44.112.76,0,True,Web Development


In [34]:
sus_2[sus_2.user_id == 341]

,start_date,end_date,created_at,cohort,program_id,time,endpoint,user_id,ip,accessed_after,is_active,program_name
date,,,,,,,,,,,,
2020-04-21,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,17:45:43,/,341,172.124.70.146,1,False,Web Development
2020-04-21,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,17:45:52,/,341,172.124.70.146,1,False,Web Development
2020-04-21,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,17:45:56,appendix,341,172.124.70.146,1,False,Web Development
2020-04-21,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,18:00:36,javascript-ii,341,172.124.70.146,1,False,Web Development
2020-04-21,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,18:23:01,/,341,172.124.70.146,1,False,Web Development
2020-04-21,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,18:24:37,appendix,341,172.124.70.146,1,False,Web Development
2020-04-21,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,18:24:38,appendix,341,172.124.70.146,1,False,Web Development
2020-04-21,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,18:26:56,/,341,172.124.70.146,1,False,Web Development
2020-04-21,2019-01-22,2019-06-04,2019-01-20 23:18:57,Zion,2.0,18:27:54,appendix,341,172.124.70.146,1,False,Web Development


### 4. Which lessons are least accessed?

There were 458 pages in the curriculum which were only accessed once. Some of them were lessons while the others were professional development pages. There was also one lesson page that seemed erroneous. The following were some of the least accessed lessons:

In [39]:
df.endpoint.value_counts(sort=True).tail(10)

6-regression/8-Project                   1
register.                                1
6.04.05_SVM                              1
content/conditionals.html                1
content/postwork/dashboard.md            1
prework/cli/07-editing-files-with-vim    1
cohorts/24/capstone-all                  1
coding-challenges                        1
java-i/control-statements-and-loops/g    1
student/850                              1
Name: endpoint, dtype: int64

In [70]:
df.endpoint.value_counts(sort=True).nsmallest(1, keep='all').count()

458

In [71]:
df.endpoint.value_counts(sort=True).nsmallest(1, keep='all')

2-storytelling/1-overview/www.qlik.com                                                                                                                                                                                                                                     1
help                                                                                                                                                                                                                                                                       1
html-css/img/favicon.ico                                                                                                                                                                                                                                                   1
slides/form                                                                                                                                                                                      

### 5. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
